## Loading Model

In [11]:
# Importing libraries
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [12]:
# Displaying preprocessed charity data
charity_df = pd.read_csv('charity_data_preprocessed.csv', index_col=0)
charity_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [13]:
# Split our preprocessed data into our features and target arrays
X = charity_df.drop(columns='IS_SUCCESSFUL')
y = charity_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
# Loading imported model
nn = tf.keras.models.load_model('AlphabetSoupCharity.h5')
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 80)                3520      
                                                                 
 dense_16 (Dense)            (None, 30)                2430      
                                                                 
 dense_17 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5981 (23.36 KB)
Trainable params: 5981 (23.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
# Print old model's loss and accuracy
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


268/268 - 0s - loss: 0.5592 - accuracy: 0.7248 - 344ms/epoch - 1ms/step
Loss: 0.5592098236083984, Accuracy: 0.724781334400177


## Optimization 1: Adding third layer

In [24]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train.shape[1]
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 15

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 80)                3520      
                                                                 
 dense_8 (Dense)             (None, 30)                2430      
                                                                 
 dense_9 (Dense)             (None, 30)                930       
                                                                 
 dense_10 (Dense)            (None, 1)                 31        
                                                                 
Total params: 6911 (27.00 KB)
Trainable params: 6911 (27.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
# Compiling model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [27]:
# Training model
nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5658 - accuracy: 0.7257
Epoch 2/100
804/804 [==============================] - 1s 967us/step - loss: 0.5549 - accuracy: 0.7294
Epoch 3/100
804/804 [==============================] - 1s 985us/step - loss: 0.5516 - accuracy: 0.7324
Epoch 4/100
804/804 [==============================] - 1s 970us/step - loss: 0.5504 - accuracy: 0.7332
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5492 - accuracy: 0.7334
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5486 - accuracy: 0.7342
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5478 - accuracy: 0.7324
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5473 - accuracy: 0.7350
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5468 - accuracy: 0.7346
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5463 - 

In [28]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5557 - accuracy: 0.7242 - 366ms/epoch - 1ms/step
Loss: 0.5557258129119873, Accuracy: 0.7241982221603394


## Optimization 2: Add more neurons

In [29]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train.shape[1]
hidden_nodes_layer1 =  86
hidden_nodes_layer2 = 43
hidden_nodes_layer3 = 21

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 86)                3784      
                                                                 
 dense_12 (Dense)            (None, 43)                3741      
                                                                 
 dense_13 (Dense)            (None, 43)                1892      
                                                                 
 dense_14 (Dense)            (None, 1)                 44        
                                                                 
Total params: 9461 (36.96 KB)
Trainable params: 9461 (36.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
# Compiling model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [31]:
# Training model
nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5675 - accuracy: 0.7244
Epoch 2/100
804/804 [==============================] - 1s 963us/step - loss: 0.5538 - accuracy: 0.7315
Epoch 3/100
804/804 [==============================] - 1s 990us/step - loss: 0.5525 - accuracy: 0.7308
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5506 - accuracy: 0.7326
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5491 - accuracy: 0.7330
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5484 - accuracy: 0.7336
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5476 - accuracy: 0.7345
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5475 - accuracy: 0.7328
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5470 - accuracy: 0.7349
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5466 - ac

In [32]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5561 - accuracy: 0.7247 - 495ms/epoch - 2ms/step
Loss: 0.5560692548751831, Accuracy: 0.7246647477149963


## Optimization 3: Adding more epochs

In [33]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train.shape[1]
hidden_nodes_layer1 =  86
hidden_nodes_layer2 = 43
hidden_nodes_layer3 = 21

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 86)                3784      
                                                                 
 dense_16 (Dense)            (None, 43)                3741      
                                                                 
 dense_17 (Dense)            (None, 43)                1892      
                                                                 
 dense_18 (Dense)            (None, 1)                 44        
                                                                 
Total params: 9461 (36.96 KB)
Trainable params: 9461 (36.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [34]:
# Compiling model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [35]:
# Training model
nn.fit(X_train_scaled,y_train,epochs=150)

Epoch 1/150
804/804 [==============================] - 1s 962us/step - loss: 0.5681 - accuracy: 0.7207
Epoch 2/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5560 - accuracy: 0.7299
Epoch 3/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5524 - accuracy: 0.7323
Epoch 4/150
804/804 [==============================] - 1s 952us/step - loss: 0.5516 - accuracy: 0.7323
Epoch 5/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5498 - accuracy: 0.7333
Epoch 6/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5488 - accuracy: 0.7339
Epoch 7/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5482 - accuracy: 0.7330
Epoch 8/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5473 - accuracy: 0.7348
Epoch 9/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5470 - accuracy: 0.7352
Epoch 10/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5462 - ac

In [36]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5616 - accuracy: 0.7265 - 589ms/epoch - 2ms/step
Loss: 0.561630129814148, Accuracy: 0.7265306115150452


In [37]:
nn.save('AlphabetSoupCharity_Optimization.h5')

C:\Users\leon1\anaconda3\envs\dev\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Trying a different model: Random Forest

In [45]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [55]:
rf_model = RandomForestClassifier(n_estimators=120, random_state=78)

In [56]:
rf_model = rf_model.fit(X_train_scaled, y_train)

In [57]:
predictions = rf_model.predict(X_test_scaled)

In [58]:
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.7100874635568513
